<a href="https://colab.research.google.com/github/Rominaarab/Big-Data/blob/main/Pypdf%2BRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving covid.pdf to covid.pdf


In [2]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 1.8 MB/s eta 0:00:00


In [3]:
from pypdf import PdfReader

In [5]:
pdf_filename = next(iter(uploaded))
reader = PdfReader(pdf_filename)

In [6]:
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    print(f"Page {i+1}:\n{text}\n")

Page 1:
COVID-19 is a respiratory illness caused by the SARS-CoV-2 virus, first identified in 
December 2019 in Wuhan, China. It spreads primarily through respiratory droplets when an 
infected person coughs, sneezes, or talks. The virus has led to global health, social, and 
economic challenges. Common symptoms include fever, cough, fatigue, and difficulty 
breathing, with some individuals developing severe complications such as pneumonia. 
Preventative measures include wearing masks, maintaining social distancing, frequent 
handwashing, and getting vaccinated. 
COVID-19 spreads mainly through airborne particles that are inhaled by individuals in close 
proximity to infected people. It can also spread by touching surfaces contaminated with the 
virus and then touching the face, especially the eyes, nose, or mouth. To reduce the risk of 
infection, public health experts recommend staying at least six feet away from others, 
wearing face masks, avoiding crowded indoor spaces, and practi

In [7]:
!pip install pypdf sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [8]:
all_text = ""
for page in reader.pages:
    text = page.extract_text()
    all_text += text + "\n"

In [9]:
def chunk_text(text, chunk_size=500, overlap=50):
    chunks = []
    i = 0
    while i < len(text):
        chunks.append(text[i:i+chunk_size])
        i += chunk_size - overlap
    return chunks

chunks = chunk_text(all_text)

In [10]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
import faiss

In [12]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

In [13]:
def rag_query(query, top_k=3):
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding), top_k)
    relevant_chunks = [chunks[idx] for idx in I[0]]
    return "\n\n".join(relevant_chunks)

In [14]:
#exapmle
query = "What are the main findings in the document?"
context = rag_query(query)
print("Retrieved context:\n", context)

Retrieved context:
 on, headache, or diarrhea. While most cases are mild, 
some individuals can develop severe respiratory distress, particularly those with underlying 
health conditions like heart disease, diabetes, and respiratory illnesses. Testing for COVID-
19 is crucial in identifying the infection early, especially for those experiencing symptoms or 
who have been in contact with an infected person. 
Vaccination against COVID-19 is one of the most effective strategies to control the spread 
of the virus. Sev

ers are also 
recommended to enhance immunity, especially against emerging variants of the virus. 
COVID-19 has mutated over time, leading to the emergence of new variants. Some variants, 
such as Delta and Omicron, have shown to be more transmissible than the original strain. 
The World Health Organization (WHO) and other health agencies closely monitor these 
variants to understand their potential impact on public health. Vaccines have been adapted 
to address newer strai

In [15]:
from transformers import pipeline

In [16]:
#Example with HuggingFace pipeline
qa_model = pipeline("text-generation", model="gpt2")
prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
answer = qa_model(prompt, max_length=200)[0]['generated_text']
print("RAG Answer:", answer)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


RAG Answer: Context:
on, headache, or diarrhea. While most cases are mild, 
some individuals can develop severe respiratory distress, particularly those with underlying 
health conditions like heart disease, diabetes, and respiratory illnesses. Testing for COVID-
19 is crucial in identifying the infection early, especially for those experiencing symptoms or 
who have been in contact with an infected person. 
Vaccination against COVID-19 is one of the most effective strategies to control the spread 
of the virus. Sev

ers are also 
recommended to enhance immunity, especially against emerging variants of the virus. 
COVID-19 has mutated over time, leading to the emergence of new variants. Some variants, 
such as Delta and Omicron, have shown to be more transmissible than the original strain. 
The World Health Organization (WHO) and other health agencies closely monitor these 
variants to understand their potential impact on public health. Vaccines have been adapted 
to address newer stra